In [1]:
%load_ext autoreload
%autoreload 2

home_or_office = "home"

import sys
if home_or_office =="home":
    sys.path.extend(['C:\\Code\\Libraries', 'C:/Code/Libraries'])
else:
    sys.path.extend(['C:\\Users\\troy\\Documents\\Code\\Libraries', 'C:\\Users\\troy\\Documents\\Excel Sheets', 
                     'C:/Users/troy/Documents/Code/Libraries'])

from datetime import date, datetime
import os
import numpy as np
import pandas as pd
import collections

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from tia.bbg import LocalTerminal
import QuantLib as ql
from FinAPI.QuantlibWrapper.Calendar import BBGCalendar

pio.renderers.default = "vscode"

### Bond Future Model in Quantlib

#### Goals
    * Read Conventions, Calendars and TimeZones
    * Create Quantlib Setting and dates and Custom Calendars
    * How to setup a bond
    * How to setup a basket of bonds
    * How to compute the yield of the futures
    * How to compute to the implied repo rate
    * How to compute the net basis and gross basis

    Other Additional goals

    * How to setup a flat yield curve
    * How to bootstrap a curve from bonds
    * How to compute Zspread

    Code Goals

    * Create a Wrapper to setup Quanlib 
    * Create instruments using basic inputs just like numerix
    * Validate Quantlib results using Numerix and Bloomberg.

#### Code

##### Calendars

In [52]:
# Read Conventions and Calendars from Numerix File
import os
import json

calendar_fldr = r"C:\Code\StaticDataNumerix\Calendar"

file_names = dict()

# Iterate through all files in the folder
for file_name in os.listdir(calendar_fldr):
    if os.path.isfile(os.path.join(calendar_fldr, file_name)):
        file_names[file_name.split(".")[0]] = os.path.join(calendar_fldr, file_name)

# Print the extracted file names
# for name in file_names:
#     print(name, file_names[name])
    
# Choose one file
name="NEWYORK"
file_path = file_names[name]
with open(file_path, 'r') as file:
    json_data = json.load(file)
    
print(json_data.keys())

calendar_dates = json_data['CALENDAR.%s.D' % name]["TABLE"]["DATES"]
# calendar_dates = [pd.to_datetime('1899-12-30') + pd.to_timedelta(excel_time,'D') for excel_time in calendar_dates]


dict_keys(['@VERSION', '@KEYS', 'CALENDAR.NEWYORK.D', 'CALENDAR.NEWYORK'])


In [10]:
# Use calendar object to retrieve holidays, check for weekends etc..
objCal = BBGCalendar("US").calendar
objCal.holidayList(_from=ql.Date(1,1,2023), to=ql.Date(31,12,2023))

(Date(2,1,2023),
 Date(16,1,2023),
 Date(20,2,2023),
 Date(7,4,2023),
 Date(29,5,2023),
 Date(19,6,2023),
 Date(4,7,2023),
 Date(4,9,2023),
 Date(9,10,2023),
 Date(23,11,2023),
 Date(25,12,2023))

In [3]:
# Yearfraction, count days, add days etc..

[Timestamp('1993-01-01 00:00:00'),
 Timestamp('1993-01-18 00:00:00'),
 Timestamp('1993-02-15 00:00:00'),
 Timestamp('1993-04-09 00:00:00'),
 Timestamp('1993-05-31 00:00:00'),
 Timestamp('1993-07-05 00:00:00'),
 Timestamp('1993-09-06 00:00:00'),
 Timestamp('1993-10-11 00:00:00'),
 Timestamp('1993-11-11 00:00:00'),
 Timestamp('1993-12-24 00:00:00'),
 Timestamp('1994-01-17 00:00:00'),
 Timestamp('1994-02-21 00:00:00'),
 Timestamp('1994-04-01 00:00:00'),
 Timestamp('1994-05-30 00:00:00'),
 Timestamp('1994-07-04 00:00:00'),
 Timestamp('1994-09-05 00:00:00'),
 Timestamp('1994-10-10 00:00:00'),
 Timestamp('1994-11-11 00:00:00'),
 Timestamp('1994-11-24 00:00:00'),
 Timestamp('1994-12-26 00:00:00'),
 Timestamp('1995-01-02 00:00:00'),
 Timestamp('1995-01-16 00:00:00'),
 Timestamp('1995-02-20 00:00:00'),
 Timestamp('1995-04-14 00:00:00'),
 Timestamp('1995-05-29 00:00:00'),
 Timestamp('1995-07-04 00:00:00'),
 Timestamp('1995-09-04 00:00:00'),
 Timestamp('1995-10-09 00:00:00'),
 Timestamp('1995-11-

##### Bonds

###### Bond from Scratch

In [55]:
# Bond Fut Conventions

static_vals_usbond = {
    "NAME" : ["Currency","Rounding","Ytm Rounding","Price Rounding Method","Freq","Ex Coupon Lag","Ex Coupon Cal","Basis","Ignore Leap Year","End Of Month","Front Stub","Long Stub","Pay Calendar","Pay Convention","Payment Lag","Accrual Calendar","Accrual Convention","Settlement Calendar","Settlement Lag","Notional","Compounding Frequency","Principal Pay Convention","Coupon Basis","Final Period Yield Type","Repay Notional","Yield Type","Discount Basis","Level Coupons","Cum Coupon On Ex Date","Final Period On Ex Date"],
    "VALUE" : ["USD","NEAREST-12","NEAREST-12","DEFAULT","6M","0BD","NONE","ACT/ACT(BOND)",False,True,True,False,"NewYork","F","0BD","NONE","NONE","NewYork","1BD",1000000,"6M","F","ACT/ACT(BOND)","MONEY MARKET",True,"COMPOUNDED","ACT/ACT(BOND)",True,True,True]
}

static_vals_fv = {
      "NAME" : ["Currency","Exchange","Ticker","Contract Size","Delivery Calendar","First Delivery Ref Day","First Delivery Date Lag","First Delivery Date Convention","Last Delivery Ref Day","Last Delivery Date Lag","Last Delivery Date Convention","Conversion Factor Type","Conversion Factor Yield","Conversion Factor Basis","Conversion Factor Maturity Rounding","Conversion Factor Rounding","Notional Bond Coupon","Notional Bond Freq","Notional Bond Maturity","Rounding"],
      "VALUE" : ["USD","CBOT","FV",100000,"CBOT",1,"0BD","F","END","3BD","F", float("nan"), 0.0599999999999999978,"30/360",1,"NEAREST-4",0.0599999999999999978,"6M","5Y","NEAREST-2"]
    }

static_data_fv = dict(zip(static_vals_fv["NAME"], static_vals_fv["VALUE"]))
static_data_bond = dict(zip(static_vals_usbond["NAME"], static_vals_usbond["VALUE"]))

# Define a schedule and bond
issueDate = ql.Date(31, 5, 2022)
maturityDate = ql.Date(31, 5, 2028)
coupons = [3.625/100]
cleanPrice = 98+11.875/32

tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = True
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                            bussinessConvention , dateGeneration, monthEnd)
print("scheduled dates" )
print(list(schedule.dates()))

settlementDays = 1
faceValue = 100.0
paymentConvention = ql.Unadjusted
paymentCalendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
redemption = 100.0
issueDate = ql.Date(31,8,2022)
accrualDayCounter = ql.ActualActual(ql.ActualActual.Bond,schedule)
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, accrualDayCounter, paymentConvention, redemption, issueDate, paymentCalendar)
print("Bond Cash Flows")
for c in fixedRateBond.cashflows():
    print('%20s %12f' % (c.date(), c.amount()))

# Settlement Date
print(fixedRateBond.settlementDate())

# Yield Function
yld = fixedRateBond.bondYield(cleanPrice, accrualDayCounter,ql.CompoundedThenSimple, ql.Semiannual, fixedRateBond.settlementDate(), 1.0e-12)
print(yld)

# Cashflow Functions
print("------- Cashflow Functions ---------")
print(ql.BondFunctions.nextCashFlowDate(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.nextCashFlowAmount(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.nextCouponRate(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.accrualStartDate(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.accrualEndDate(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.accrualDays(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.accrualPeriod(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.accruedPeriod(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.accruedAmount(fixedRateBond, fixedRateBond.settlementDate()))
print(ql.BondFunctions.accruedDays(fixedRateBond, fixedRateBond.settlementDate()))

# Risk Functions
print("------ Risk Functions ----------")
print(ql.BondFunctions.duration(fixedRateBond, yld, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual, ql.Duration.Simple, 
                                fixedRateBond.settlementDate()))
print(ql.BondFunctions.duration(fixedRateBond, yld, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual, ql.Duration.Modified, 
                                fixedRateBond.settlementDate()))
print(ql.BondFunctions.basisPointValue(fixedRateBond, yld, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual, fixedRateBond.settlementDate()))
print(ql.BondFunctions.convexity(fixedRateBond, yld, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual, fixedRateBond.settlementDate()))

# Risk from first principles
yieldVal = ql.InterestRate(yld, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual)
ql.BondFunctions.duration(fixedRateBond, yieldVal, ql.Duration.Modified, fixedRateBond.settlementDate())

P = fixedRateBond.cleanPrice(yld, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual, fixedRateBond.settlementDate())
dP_U = fixedRateBond.cleanPrice(yld+0.0001, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual, fixedRateBond.settlementDate())
dP_D = fixedRateBond.cleanPrice(yld-0.0001, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual, fixedRateBond.settlementDate())
dP_dy = (dP_U - dP_D) / (0.0002)
d2P_dy2 = (dP_U + dP_D - 2 * P)/(0.01)**2
print("DV01", dP_dy , "Price", P, "Modified Duration", dP_dy/ P, " Convexity", d2P_dy2, "Up DV01", (P-dP_U)/0.0001, "Down DV01" , (dP_D - P)/0.0001)

macD = ql.BondFunctions.duration(fixedRateBond, yld, accrualDayCounter, ql.CompoundedThenSimple, ql.Semiannual, ql.Duration.Simple, 
                                 fixedRateBond.settlementDate())
print("macD", macD, "modifedD - calc", 4.547 / (1 + yld/2), "DV01 - calc", 4.547 / (1 + yld/2) * cleanPrice)

scheduled dates
[Date(31,5,2022), Date(30,11,2022), Date(31,5,2023), Date(30,11,2023), Date(31,5,2024), Date(30,11,2024), Date(31,5,2025), Date(30,11,2025), Date(31,5,2026), Date(30,11,2026), Date(31,5,2027), Date(30,11,2027), Date(31,5,2028)]
Bond Cash Flows
 November 30th, 2022     1.812500
      May 31st, 2023     1.812500
 November 30th, 2023     1.812500
      May 31st, 2024     1.812500
 November 30th, 2024     1.812500
      May 31st, 2025     1.812500
 November 30th, 2025     1.812500
      May 31st, 2026     1.812500
 November 30th, 2026     1.812500
      May 31st, 2027     1.812500
 November 30th, 2027     1.812500
      May 31st, 2028     1.812500
      May 31st, 2028   100.000000
July 3rd, 2023
0.03993039773538477
------- Cashflow Functions ---------
November 30th, 2023
1.8124999999999947
0.03625
May 31st, 2023
November 30th, 2023
183
0.5
0.09016393442622951
0.3268442622950785
33
------ Risk Functions ----------
4.528066646563029
3.805604797913669
-0.037560384929529025
30.

###### Bond Quantlib Object

In [69]:
from FinAPI.QuantlibWrapper.Bonds import USTreasury

""" Bond Objmethod

    .settlementValue(cleanPrice)  == cleanPrice + Bond.accruedAmount()
    
"""

In [99]:
issueDate = ql.Date(31, 5, 2023)
maturityDate = ql.Date(31, 5, 2028)
coupons = 3.625/100
cleanPrice = 97+21.125/32
exp_yld = 4.156132

objBond: USTreasury = USTreasury(coupons, maturityDate, issueDate)
yld = objBond.street_yield(cleanPrice)
print(yld, objBond.settlementDate(), objBond.duration(yld), objBond.modifiedDuration(yld), objBond.DV01(yld), objBond.convexity(yld))
objBond.nextCashFlowDate(), objBond.nextCashFlowAmount(), objBond.couponRate, objBond.settlementValue(cleanPrice)
objBond.maturityDate - objBond.calendar.

0.04156643367529615 July 5th, 2023 4.5211897686537394 4.429138032520003 4.336428165512984 22.342792675544842


AttributeError: 'function' object has no attribute 'today'

In [ ]:
def create_tsy_security(bond_issue_date, bond_maturity_date, coupon_rate, coupon_frequency=ql.Period(6, ql.Months), day_count=ql.ActualActual(),
                        calendar= ql.UnitedStates()
                        ):
    face_value = 100.
    settlement_days = 0
    
    schedule = ql.Schedule(bond_issue_date, bond_maturity_date, coupon_frequency, calendar, ql.ModifiedFollowing, ql.ModifiedFollowing, 
                           ql.DateGeneration.Forward, False)

    security = ql.FixedRateBond(settlement_days,face_value, schedule, [coupon_rate], day_count)
    return security

In [2]:


# Set the evaluation date
evaluation_date = ql.Date(15, 6, 2023)
ql.Settings.instance().evaluationDate = evaluation_date

# more Settings
day_count = ql.ActualActual()
calendar = ql.UnitedStates()
bussiness_convention = ql.Following
end_of_month = False
settlement_days = 0
face_amount = 100
coupon_frequency = ql.Period(ql.Semiannual)

# Set up the yield curve
risk_free_rate = 0.05
yield_curve = ql.FlatForward(evaluation_date, risk_free_rate, ql.Actual365Fixed())

# Set up the bond future contract specifications
contract_months = [ql.Date(1, 3, 2024), ql.Date(1, 6, 2024), ql.Date(1, 9, 2024)]
last_trading_date = contract_months[-1]
future_delivery_date = ql.IMM.nextDate(last_trading_date)
conversion_factor = 0.8  # Modify this with the appropriate conversion factor for the specific bond future

# Set up the bond future
bond_future = ql.BondFuture(future_delivery_date, contract_months, conversion_factor)

# Set up the pricing engine
engine = ql.DiscountingBondEngine(yield_curve)

# Price the bond future
bond_future.setPricingEngine(engine)
bond_future_price = bond_future.NPV()
print("Bond Future Price:", bond_future_price)

# Access the bond future's conversion factors
conversion_factors = bond_future.conversionFactors()

# Access the bond future's cheapest-to-deliver bond
cheapest_to_deliver = bond_future.cheapestToDeliver()

# Access the bond future's implied repo rate
implied_repo_rate = bond_future.impliedRepoRate(bond_future_price, yield_curve)
print("Implied Repo Rate:", implied_repo_rate)

AttributeError: module 'QuantLib' has no attribute 'BondFuture'

In [72]:
98+11.875/32

98.37109375